## Process Based Concurrency and creating and starting child processes

A couple of items to note before working with concurrency, parellism, threats and processes (each application is a process and has a single default main thread - task).
- `Concurrency` means doing things out of order i.e. executing tasks out of order
- `Parellelism` executing two tasks at the same time
- `GIL` threading is limited by the Global Interpretor lock
- `IPC` inter-process connectivity

### Multiprocessing vs threading

Similarities:

1. Both Modules are used for concurrency 
2. Similar APIs
3. Support for concurrency primitives in Python

Differences: 
1. Native threads vs Native processes
2. Shared memory vs IPCs (Inter process communications)
3. Limited vs Full Parallelism (GIL)

Generally, use the `multiprocessing` package for CPU bound tasks and not for input / output (IO) bound tasks. 

### Creating and starting a process

- `Main process` = default process created to execute a Python program, has the name *MainProcess*.
- `Main thread` = default thread created by a main process in a Python program, has the name *MainThread*


In [7]:
from time import sleep
from multiprocessing import Process
help(Process)

Help on class Process in module multiprocessing.context:

class Process(multiprocessing.process.BaseProcess)
 |  Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
 |  
 |  Method resolution order:
 |      Process
 |      multiprocessing.process.BaseProcess
 |      builtins.object
 |  
 |  Methods inherited from multiprocessing.process.BaseProcess:
 |  
 |  __init__(self, group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  close(self)
 |      Close the Process object.
 |      
 |      This method releases resources held by the Process object.  It is
 |      an error to call this method if the child process is still running.
 |  
 |  is_alive(self)
 |      Return whether process is alive
 |  
 |  join(self, timeout=None)
 |      Wait until child process terminates
 |  
 |  kill(self)
 |      Terminate process;

In [8]:
%%writefile create_process.py
from time import sleep
from multiprocessing import Process

def task(): 
    sleep(1)
    print('This is from another process', flush=True)

if __name__=='__main__':
    process = Process(target=task)
    process.start()
    print('Waiting for the process to finish')
    process.join()

Overwriting create_process.py


## Extend the process class

In [9]:
%%writefile process_class_extend.py
from time import sleep
from multiprocessing import Process

class CustomProcess(Process):
    def run(self):
        sleep(1)
        print('This is another process', flush=True)
    def __str__(self):
        return 'This is another process'

# Create entrypoint script in Python
if __name__ == '__main__':
    process = CustomProcess()
    print(process)
    process.start()
    print('Waiting for the process to finish')
    process.join()

Overwriting process_class_extend.py


## Configure and interact with processes

The below shows how to name a process:

In [10]:
%%writefile name_process.py
from multiprocessing import Process

if __name__=='__main__':
    process = Process(name='FunkyProcess')
    print(process.name)

Overwriting name_process.py


Configure whether the process is a Daemon or not:

In [11]:
%%writefile check_daemon_proc.py
from multiprocessing import Process

if __name__== '__main__':
    proc = Process(daemon=True)
    print(proc.daemon)

Overwriting check_daemon_proc.py


### Query the status of a process

Query the process and the PID:

In [12]:
%%writefile query_proc_pid.py
from multiprocessing import Process

if __name__ =='__main__':
    process = Process()
    print(process.pid)
    process.start()
    print(process.pid)

Overwriting query_proc_pid.py


You can also check if a process is alive or dead:

In [13]:
%%writefile check_is_alive_proc.py
from multiprocessing import Process

if __name__ == '__main__':
    process = Process()
    print(process.is_alive())

Overwriting check_is_alive_proc.py


Child process exit code example:

In [14]:
%%writefile child_proc_exit_code.py
from time import sleep
from multiprocessing import Process

def task():
    sleep(1)

if __name__ == '__main__':
    process = Process(target=task)
    print(process.exitcode)
    process.start()
    print(process.exitcode)
    process.join()
    print(process.exitcode)

Overwriting child_proc_exit_code.py


### Terminate or kill a process

In [ ]:
# process.terminate()
# process.kill()

### Get the current, parent and child process

In [15]:
%%writefile get_current_proc.py
from multiprocessing import current_process

if __name__=='__main__':
    process = current_process()
    print(process)

Writing get_current_proc.py


In [16]:
%%writefile get_parent_proc.py
from multiprocessing import parent_process

if __name__=='__main__':
    process = parent_process()
    print(process)

Writing get_parent_proc.py


In [17]:
%%writefile get_child_proc.py
from time import sleep
from multiprocessing import active_children
from multiprocessing import Process

def task(): 
    sleep(1)

if __name__=='__main__':
    processes = [Process(target=task) for _ in range(5)]
    # Start the child processes
    for process in processes:
        process.start()

    children = active_children()
    print(f'Active children count:{len(children)}')

    for child in children:
        print(child)

Writing get_child_proc.py


## Synchronise and coordinate process

### Mutex lock to protect critical sections

In [18]:
%%writefile mutex_locking.py
from time import sleep
from random import random
from multiprocessing import Process
from multiprocessing import Lock

def task(shared_lock, ident, value):
    with shared_lock:
        print(f'*{ident} got a lock, sleeping {value}')
        sleep(value)

# Protect the entry point
if __name__=='__main__':
    lock = Lock()
    # Create a number of processes with different args
    processes = [Process(target=task,
                        args=(lock, i, random())) for i in range(10)]
    
    for process in processes:
        process.start()

    for process in processes:
        process.join()

Writing mutex_locking.py


## Semaphore to limit access to shared process

In [19]:
%%writefile semaphore_shared_access.py
from time import sleep
from random import random
from multiprocessing import Process, Semaphore

# Task to run on the CPU
def task(shared_semaphore, ident):
    with shared_semaphore:
        val = random()
        sleep(val)
        print(f'Process {ident} got {val}', flush=True)

if __name__ =='__main__':
    # Create the shared semaphore
    semap = Semaphore(2)
    processes = [Process(target=task, args=(semap, i)) for i in range(100)]

    # Start Child processes
    for process in processes:
        process.start()

    # wait for child processes to finish
    for process in processes:
        process.join()

Writing semaphore_shared_access.py


## Signal between processes using an Event

In [21]:
%%writefile signal_proc_btw_event.py
from time import sleep
from random import random
from multiprocessing import Process, Event

def task(shared_event, number):
    print(f'Process {number} waiting...', flush=True)
    shared_event.wait()
    value = random()
    sleep(value)
    print(f'Process {number} got {value}', flush=True)

if __name__ == '__main__':
    event = Event()
    processes = [Process(target=task, args=(event, i)) for i in range(5)]

    for process in processes:
        process.start()
    
    print('Main process blocking...')
    sleep(2)
    # Trigger all child processes
    event.set()

    for process in processes: 
        process.join()


Overwriting signal_proc_btw_event.py
